# <span style='color:blue'> Hyperparameter optimization for ML classifiers </span>

The present `Jupyter Notebook` contains the hyperparamter optimization by randomized search of the ML classifiers implemented.

## Import Essentials

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression


In [2]:
# Import functions from customized modules
from pathlib import Path
import sys

PROJECT_DIR = list(Path("..").absolute().parents)[0]
sys.path.append(str(PROJECT_DIR) + '/Scripts/')

from preprocessing import create_mol
from classification import iter_rand_search

## Define file names

In [3]:
# File names and sheet names
descriptors_file = './Data/BTZs_full_descriptors_final.xlsx'
data_matrix = 'general_data_matrix'
desc = 'descriptors'

## Load data  

In [4]:
# Load data set with SMILES
BTZs = pd.read_excel(descriptors_file, sheet_name=data_matrix)

# Load set of descriptors 
desc = pd.read_excel(descriptors_file, sheet_name=desc)

In [5]:
BTZs.shape

(96, 7)

In [6]:
# Check coincidence between dataframes
for i, j in zip(BTZs.iloc[:, 0], desc.iloc[:, 0]):
    if i != j:
        print('Error')

#### Calculate Fingerprints

Here a locally created function using RDKit is employed.

In [7]:
# Start from the Smiles string stored in BTZs dataframe
BTZs_morgan = BTZs.copy()
# Apply create_mol
create_mol(BTZs_morgan, 'ecfp4', n_bits=2048)

In [8]:
# Start from the Smiles string stored in BTZs dataframe
BTZs_rdkit = BTZs.copy()
# Apply create_mol --> add columns to the dataframe with mol and fp
create_mol(BTZs_rdkit, 'rdk5', n_bits=2048)

#### Prepare data sets

In [9]:
# Create X data sets from descriptors and molecular fingerprints
Xdesc = desc.iloc[:, 1:] 
print('X descriptors shape {}'.format(Xdesc.shape))

Xmorg = pd.DataFrame(BTZs_morgan.np_bv.to_list(), index=BTZs_morgan.index)
print('X Morgan shape {}'.format(Xmorg.shape))

Xrdk = pd.DataFrame(BTZs_rdkit.np_bv.to_list(), index=BTZs_rdkit.index)
print('X RDKit shape {}'.format(Xrdk.shape))

X descriptors shape (96, 227)
X Morgan shape (96, 2048)
X RDKit shape (96, 2048)


In [10]:
y = BTZs.Class

print('y shape {}'.format(y.shape))

y shape (96,)


## Hyperparameter optimization 

Three different random seeds were used for splitting the data set into training and test set and training the models.

In [11]:
# Define a list of random seeds
seed = [57, 31, 86]

In [12]:
# Define base models
model_rf = RandomForestClassifier(random_state=21)
model_knn = KNeighborsClassifier()
model_svm = SVC()
model_ada = AdaBoostClassifier(random_state=21)

In [13]:
# Define hyperparamaters to optimize for RF
param_rf = {'n_estimators': np.linspace(5, 1000, 200).astype('int'), 
            'min_samples_split': np.linspace(2, 16, 15).astype('int'),
            'min_samples_leaf': np.linspace(1, 10, 10).astype('int'),
            'max_features': np.linspace(0.05, 1, 20).astype('float')}


In [14]:
# Define hyperparamaters to optimize for kNN
param_knn = {'n_neighbors': np.linspace(1, 20, 20).astype('int'),
             'leaf_size': np.linspace(1, 10, 10).astype('int'),
             'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
             'weights': ['uniform', 'distance']}

In [15]:
# Define hyperparamaters to optimize for SVM
param_svm = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 
             'gamma': [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1],
             'C': np.linspace(1, 100, 46).astype('float')/10}


In [16]:
# Define hyperparamaters to optimize for AdaBoost
e0 = DecisionTreeClassifier(max_depth=1, random_state=21)
e1 = DecisionTreeClassifier(max_depth=2, random_state=21)
e2 = DecisionTreeClassifier(max_depth=3, random_state=21)
e3 = LogisticRegression(penalty='none', random_state=21)
e4 = LogisticRegression(penalty='l2', random_state=21)


param_ada = {'n_estimators': np.linspace(5, 500, 100).astype('int'), 
             'learning_rate': np.concatenate((np.linspace(0.1, 2.1, 21), 
                              np.linspace(0.001, 0.1, 31)), axis=0).astype('float'),
             'base_estimator': [e0, e1, e2, e3, e4]}


In [17]:
# Compile data sets
features = {}
features['label'] = ['desc', 'morgan', 'rdkit']
features['values'] = [Xdesc, Xmorg, Xrdk]
features['preprocessing'] = ['filter', 'split', 'split']

In [18]:
# Compile models and dictionaries with hyperparameters for search
models = {}
models['label'] = ['rf', 'knn', 'svm', 'ada']
models['algorithm'] = [model_rf, model_knn, model_svm, model_ada]
models['hyperparameters'] = [param_rf, param_knn, param_svm, param_ada]
models['iterations'] = [2000, 1200, 1000, 2000]

In [20]:
# Run a loop for searching best hyperparameters combination for each algorithm and data set

results_hyperparam = []
for i in range(len(features['label'])):
    for j in range(len(models['label'])):
        lab = 'result_rd_{}_{}'.format(models['label'][j], features['label'][i])
        ml = models['algorithm'][j]
        X = features['values'][i]
        preproc = features['preprocessing'][i]
        param = models['hyperparameters'][j]
        niter = models['iterations'][j]
        print('Running {} with {}'.format(models['label'][j], features['label'][i]))
        result = iter_rand_search(ml, X, y, preproc, seed, niter, param)
        results_hyperparam.append([lab, result])

Running rf with desc
Fitting 15 folds for each of 2000 candidates, totalling 30000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done  72 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 322 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done 672 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done 1122 tasks      | elapsed:   20.9s
[Parallel(n_jobs=-1)]: Done 1672 tasks      | elapsed:   32.4s
[Parallel(n_jobs=-1)]: Done 2322 tasks      | elapsed:   44.3s
[Parallel(n_jobs=-1)]: Done 3072 tasks      | elapsed:   56.6s
[Parallel(n_jobs=-1)]: Done 3922 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 4872 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 5922 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 7072 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 8322 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 9672 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 11122 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 12672 tasks    

Fitting 15 folds for each of 2000 candidates, totalling 30000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done  72 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 322 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 672 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done 1122 tasks      | elapsed:   21.4s
[Parallel(n_jobs=-1)]: Done 1672 tasks      | elapsed:   33.1s
[Parallel(n_jobs=-1)]: Done 2322 tasks      | elapsed:   45.2s
[Parallel(n_jobs=-1)]: Done 3072 tasks      | elapsed:   57.7s
[Parallel(n_jobs=-1)]: Done 3922 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 4872 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 5922 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 7072 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 8322 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 9672 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 11122 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 12672 tasks    

Fitting 15 folds for each of 2000 candidates, totalling 30000 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done  73 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 432 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 800 tasks      | elapsed:   16.9s
[Parallel(n_jobs=-1)]: Done 1250 tasks      | elapsed:   27.4s
[Parallel(n_jobs=-1)]: Done 1800 tasks      | elapsed:   39.9s
[Parallel(n_jobs=-1)]: Done 2450 tasks      | elapsed:   53.4s
[Parallel(n_jobs=-1)]: Done 3200 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 4050 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 5000 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 6050 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 7200 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 8450 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 9800 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 11250 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 12800 tasks    

Fitting 15 folds for each of 2000 candidates, totalling 30000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done  72 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done 800 tasks      | elapsed:   17.4s
[Parallel(n_jobs=-1)]: Done 1250 tasks      | elapsed:   28.6s
[Parallel(n_jobs=-1)]: Done 1800 tasks      | elapsed:   41.0s
[Parallel(n_jobs=-1)]: Done 2450 tasks      | elapsed:   56.0s
[Parallel(n_jobs=-1)]: Done 3200 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 4050 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 5000 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 6050 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 7200 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 8450 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 9800 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 11250 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 12800 tasks    

Fitting 15 folds for each of 2000 candidates, totalling 30000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done  73 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 428 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 800 tasks      | elapsed:   16.7s
[Parallel(n_jobs=-1)]: Done 1250 tasks      | elapsed:   27.5s
[Parallel(n_jobs=-1)]: Done 1800 tasks      | elapsed:   39.8s
[Parallel(n_jobs=-1)]: Done 2450 tasks      | elapsed:   53.5s
[Parallel(n_jobs=-1)]: Done 3200 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 4050 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 5000 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 6050 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 7200 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 8450 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 9800 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 11250 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 12800 tasks    

In [21]:
results_hyperparam[0][1]

,n_estimators,min_samples_split,min_samples_leaf,max_features,score,Accuracy,f1_score,Acc_CV,f1_CV
57,775,3,1,0.10,0.787513,0.833333,0.818182,0.744127,0.757022
31,830,2,1,0.05,0.833266,0.833333,0.846154,0.768889,0.786207
86,10,4,1,0.60,0.777079,0.791667,0.736842,0.691746,0.745638


In [23]:
# Use the created list to export results as csv files
for i in results_hyperparam:
    i[1].to_csv('./Results/' + i[0] + '.csv')